In [1]:

import pandas as pd
import numpy as np
import yfinance as yf
import math
from scipy import stats


In [2]:
tickers=pd.read_csv(r"C:\Users\KIIT\Desktop\Code\top_50_indian_stocks.csv")
tickers.head()

,Ticker,Company Name
0,RELIANCE.NS,Reliance Industries
1,TCS.NS,Tata Consultancy Services
2,HDFCBANK.NS,HDFC Bank
3,INFY.NS,Infosys
4,ICICIBANK.NS,ICICI Bank


## weighted scoring model for finding the final score

In [ ]:
def create_dividend_df(tickers):
    columns = [
        "Ticker", 
        "Dividend Yield(%)",  
        "Dividend Rate",
        "Payout Ratio(%)", 
        "5 Year Avg Dividend Yield(%)", 
        "Earning Growth(%)"
        
    ]
    
    dividend_df = pd.DataFrame(columns=columns)
    
    for stock in tickers:
        ticker = yf.Ticker(stock)
        info = ticker.info 
        
        dividend_yield = info.get("dividendYield", np.nan) * 100 if info.get("dividendYield") else np.nan
        dividend_rate = info.get("dividendRate", np.nan) 
        payout_ratio = info.get("payoutRatio", np.nan) * 100 if info.get("payoutRatio") else np.nan
        five_year_avg_dividend_yield = info.get("fiveYearAvgDividendYield", np.nan) * 100 if info.get("fiveYearAvgDividendYield") else np.nan
        earning_growth = info.get("earningsGrowth", np.nan) * 100 if info.get("earningsGrowth") else np.nan
        
        dividend_df.loc[len(dividend_df)] = [stock, dividend_yield, dividend_rate, payout_ratio, five_year_avg_dividend_yield, earning_growth]
        
    # we have got our df with dividend information
    
    numeric_cols = [
        "Dividend Yield(%)", 
        "Dividend Rate",
        "Payout Ratio(%)", 
        "5 Year Avg Dividend Yield(%)", 
        "Earning Growth(%)"
    ]
    

    for col in numeric_cols:
        if col == "Payout Ratio":
            dividend_df[col+ " Normalised"] = 1 - ((dividend_df[col] - dividend_df[col].min()) / (dividend_df[col].max() - dividend_df[col].min()))
        else:
            dividend_df[col+ " Normalised"] = (dividend_df[col] - dividend_df[col].min()) / (dividend_df[col].max() - dividend_df[col].min())
        
    return dividend_df

In [4]:
tickers_list = tickers["Ticker"].values.tolist()
dividend_df = create_dividend_df(tickers_list)
dividend_df

,Ticker,Dividend Yield(%),Dividend Rate,Payout Ratio(%),5 Year Avg Dividend Yield(%),Earning Growth(%),Dividend Yield(%) Normalised,Dividend Rate Normalised,Payout Ratio(%) Normalised,5 Year Avg Dividend Yield(%) Normalised,Earning Growth(%) Normalised
0,RELIANCE.NS,36.0,5.0,8.300000,44.0,78.4,0.037126,0.029851,0.046889,0.042889,0.303568
1,TCS.NS,201.0,61.0,52.140000,131.0,6.0,0.234731,0.447761,0.357878,0.141084,0.157630
2,HDFCBANK.NS,109.0,22.0,25.150000,67.0,-2.2,0.124551,0.156716,0.166418,0.068849,0.141101
3,INFY.NS,417.0,63.0,65.900004,216.0,5.6,0.493413,0.462687,0.455487,0.237020,0.156823
4,ICICIBANK.NS,74.0,11.0,13.600000,99.0,14.6,0.082635,0.074627,0.084486,0.104966,0.174965
5,HINDUNILVR.NS,190.0,48.0,94.879997,154.0,5.6,0.221557,0.350746,0.661063,0.167043,0.156823
6,SBIN.NS,200.0,15.9,15.760000,139.0,-8.3,0.233533,0.111194,0.099808,0.150113,0.128805
7,BAJFINANCE.NS,50.0,4.4,15.680000,24.0,19.7,0.053892,0.025373,0.099241,0.020316,0.185245
8,BHARTIARTL.NS,84.0,16.0,14.280000,80.0,423.9,0.094611,0.111940,0.089310,0.083521,1.000000
9,ITC.NS,381.0,15.7,88.830000,406.0,284.3,0.450299,0.109701,0.618146,0.451467,0.718605


In [ ]:
weights={
    "Dividend Yield(%) Normalised": 0.3,
    "Dividend Rate Normalised": 0.2,
    "Payout Ratio(%) Normalised": 0.2,
    "5 Year Avg Dividend Yield(%) Normalised": 0.2,
    "Earning Growth(%) Normalised": 0.1
} #weights are based on Empirical Testing / Backtesting
dividend_df['Dividend score'] =dividend_df[[col for col in weights.keys()]].mul(list(weights.values())).sum(axis=1)
dividend_df


,Ticker,Dividend Yield(%),Dividend Rate,Payout Ratio(%),5 Year Avg Dividend Yield(%),Earning Growth(%),Dividend Yield(%) Normalised,Dividend Rate Normalised,Payout Ratio(%) Normalised,5 Year Avg Dividend Yield(%) Normalised,Earning Growth(%) Normalised,Dividend score
0,RELIANCE.NS,36.0,5.0,8.300000,44.0,78.4,0.037126,0.029851,0.046889,0.042889,0.303568,0.065420
1,TCS.NS,201.0,61.0,52.140000,131.0,6.0,0.234731,0.447761,0.357878,0.141084,0.157630,0.275527
2,HDFCBANK.NS,109.0,22.0,25.150000,67.0,-2.2,0.124551,0.156716,0.166418,0.068849,0.141101,0.129872
3,INFY.NS,417.0,63.0,65.900004,216.0,5.6,0.493413,0.462687,0.455487,0.237020,0.156823,0.394745
4,ICICIBANK.NS,74.0,11.0,13.600000,99.0,14.6,0.082635,0.074627,0.084486,0.104966,0.174965,0.095103
5,HINDUNILVR.NS,190.0,48.0,94.879997,154.0,5.6,0.221557,0.350746,0.661063,0.167043,0.156823,0.317920
6,SBIN.NS,200.0,15.9,15.760000,139.0,-8.3,0.233533,0.111194,0.099808,0.150113,0.128805,0.155163
7,BAJFINANCE.NS,50.0,4.4,15.680000,24.0,19.7,0.053892,0.025373,0.099241,0.020316,0.185245,0.063678
8,BHARTIARTL.NS,84.0,16.0,14.280000,80.0,423.9,0.094611,0.111940,0.089310,0.083521,1.000000,0.185338
9,ITC.NS,381.0,15.7,88.830000,406.0,284.3,0.450299,0.109701,0.618146,0.451467,0.718605,0.442813


In [ ]:
dividend_df = dividend_df.sort_values(by = "Dividend score", ascending = False) #why false?
dividend_df.head(10)

,Ticker,Dividend Yield(%),Dividend Rate,Payout Ratio(%),5 Year Avg Dividend Yield(%),Earning Growth(%),Dividend Yield(%) Normalised,Dividend Rate Normalised,Payout Ratio(%) Normalised,5 Year Avg Dividend Yield(%) Normalised,Earning Growth(%) Normalised,Dividend score
30,COALINDIA.NS,840.0,31.6,49.000000,892.0,-20.2,1.000000,0.228358,0.335603,1.000000,0.104818,0.623274
42,HEROMOTOCO.NS,305.0,130.0,64.019996,310.0,24.2,0.359281,0.962687,0.442151,0.343115,0.194316,0.476806
20,ONGC.NS,726.0,17.5,46.880000,517.0,-33.4,0.863473,0.123134,0.320565,0.576749,0.078210,0.470953
28,TECHM.NS,307.0,45.0,142.660000,216.0,34.0,0.361677,0.328358,1.000000,0.237020,0.214070,0.442986
9,ITC.NS,381.0,15.7,88.830000,406.0,284.3,0.450299,0.109701,0.618146,0.451467,0.718605,0.442813
22,POWERGRID.NS,515.0,15.0,65.639997,504.0,-2.5,0.610778,0.104478,0.453643,0.562077,0.140496,0.421323
13,HCLTECH.NS,368.0,54.0,86.300004,313.0,-7.7,0.434731,0.395522,0.600199,0.346501,0.130014,0.411865
3,INFY.NS,417.0,63.0,65.900004,216.0,5.6,0.493413,0.462687,0.455487,0.237020,0.156823,0.394745
45,IOC.NS,206.0,3.0,70.919997,725.0,57.8,0.240719,0.014925,0.491097,0.811512,0.262044,0.361927
49,HDFCAMC.NS,159.0,90.0,74.140000,160.0,23.6,0.184431,0.664179,0.513939,0.173815,0.193106,0.345027
